In [13]:
import cv2
import cvzone
import math
import numpy as np
from ultralytics import YOLO
from sort import *
import time

In [14]:
video_path = r'2_traffic.mp4'
cap = cv2.VideoCapture(video_path)
model = YOLO('yolov8n.pt')

In [15]:
classnames = []
with open('classes.txt', 'r') as f:
    classnames = f.read().splitlines()


In [16]:
road_zoneA = np.array([[250, 500], [670, 500], [670, 710], [250, 710], [250, 500],], np.int32)
                       # t-left    # t-right     # b-right    # b-left  # t-left
road_zoneB = np.array([[710, 500], [1100, 500], [1100, 710], [710, 710], [710, 500]], np.int32)

road_zoneC = np.array([[920, 300], [1250, 300], [1350, 490],  [1020, 490] , [920, 300] ], np.int32)

zoneA_Line = np.array([road_zoneA[0],road_zoneA[1]]).reshape(-1)
zoneB_Line = np.array([road_zoneB[0],road_zoneB[1]]).reshape(-1)
zoneC_Line = np.array([road_zoneC[0],road_zoneC[1]]).reshape(-1)


In [17]:
tracker = Sort()
zoneAcounter = []
zoneBcounter = []
zoneCcounter = []

In [18]:
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1366,768))
    results = model(frame)
    current_detections = np.empty([0,5])

    for info in results:
        parameters = info.boxes
        for box in parameters:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1
            confidence = box.conf[0]
            class_detect = box.cls[0]
            class_detect = int(class_detect)
            class_detect = classnames[class_detect]
            conf = math.ceil(confidence * 100)
            cvzone.putTextRect(frame, f'{class_detect}', [x1 + 8, y1 - 12], thickness=2, scale=1)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            if class_detect == 'car' or class_detect == 'truck' or class_detect == 'bus'\
                    and conf > 60:
                detections = np.array([x1,y1,x2,y2,conf])
                current_detections = np.vstack([current_detections,detections])

    cv2.polylines(frame,[road_zoneA], isClosed=False, color=(0, 0, 255), thickness=8)
    cv2.polylines(frame, [road_zoneB], isClosed=False, color=(0, 255, 255), thickness=8)
    cv2.polylines(frame, [road_zoneC], isClosed=False, color=(255,0, 0), thickness=8)

    track_results = tracker.update(current_detections)
    for result in track_results:
        x1,y1,x2,y2,id = result
        x1,y1,x2,y2,id = int(x1),int(y1),int(x2),int(y2),int(id)
        w, h = x2 - x1, y2 - y1
        cx, cy = x1 + w // 2, y1 + h // 2 -40

 
        if zoneA_Line[0] < cx < zoneA_Line[2] and zoneA_Line[1] - 20 < cy < zoneA_Line[3] + 20:
            if zoneAcounter.count(id) == 0:
                zoneAcounter.append(id)

        if zoneB_Line[0] < cx < zoneB_Line[2] and zoneB_Line[1] - 20 < cy < zoneB_Line[3] + 20:
            if zoneBcounter.count(id) == 0:
                zoneBcounter.append(id)

        if zoneC_Line[0] < cx < zoneC_Line[2] and zoneC_Line[1] - 20 < cy < zoneC_Line[3] + 20:
            if zoneCcounter.count(id) == 0:
                zoneCcounter.append(id)

        cv2.circle(frame,(870,40),15,(0,0,255),-1)
        cv2.circle(frame,(870,100),15,(0,255,255),-1)
        cv2.circle(frame,(870,160),15,(255,0,0),-1)
        cvzone.putTextRect(frame, f'LANE A Vehicles ={len(zoneAcounter)}', [900, 50], thickness=2, scale=2.0, border=2)
        cvzone.putTextRect(frame, f'LANE B Vehicles ={len(zoneBcounter)}', [900, 110], thickness=2, scale=2.0, border=2)
        cvzone.putTextRect(frame, f'LANE C Vehicles ={len(zoneCcounter)}', [900, 170], thickness=2, scale=2.0, border=2)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


0: 384x640 22 cars, 1 bus, 2 trucks, 881.5ms
Speed: 11.6ms preprocess, 881.5ms inference, 8.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 2 trucks, 493.3ms
Speed: 69.1ms preprocess, 493.3ms inference, 6.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 2 trucks, 377.3ms
Speed: 10.6ms preprocess, 377.3ms inference, 6.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 cars, 1 bus, 2 trucks, 345.7ms
Speed: 8.3ms preprocess, 345.7ms inference, 5.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 cars, 1 bus, 2 trucks, 349.8ms
Speed: 10.9ms preprocess, 349.8ms inference, 5.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 cars, 1 bus, 2 trucks, 340.4ms
Speed: 8.5ms preprocess, 340.4ms inference, 6.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 1 truck, 342.8ms
Speed: 8.3ms preprocess, 342.8ms inference, 3.5ms postprocess per image at shape (1, 3,